# Annotating Data & Training Custom NER model for Suicide Risk Prediction

In this tutorial, we'll see how we can easily pre-annotate data using pre-defined vocabulary / key-word matching, and upload them as pre-annotations to NLPLab.

We'll be using the AnnotationLab module to create, configure, export and import projects with minimal code.

Note: The Annotation Lab module is available in Spark NLP for Healthcare 4.2.2+.

## Following are the main steps in this exercise:

### 1. Using string matching and existing off-the-shelf vocabularies, create a simple pipeline to get rudimentary results.
### 2. Upload the initial results to Annotation Lab, annotate, and download annotations.
### 3. Train an NER model on the annotated data to achieve better performance.

## 0. Initial Configurations

In [0]:
import pandas as pd
import os

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *

from pyspark.ml import Pipeline, PipelineModel
from sparknlp.training import CoNLL

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 4.2.4
sparknlp_jsl.version : 4.2.3
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.6:7077 
 AppName 
 Databricks Shell

For this exercise, we define two different layers of solutions:
1. Bronze: Text Matcher based rudimentary results - that will be uploaded in annotation lab and refined.
2. Silver: After annotating the documents properly in Annotation Lab, train an NER model, get results

In [0]:

delta_bronze_path='/FileStore/HLS/nlp/delta/bronze/'

dbutils.fs.mkdirs(delta_bronze_path)

os.environ['delta_bronze_path']=f'/dbfs{delta_bronze_path}'

delta_silver_path='/FileStore/HLS/nlp/delta/silver/'

dbutils.fs.mkdirs(delta_silver_path)

os.environ['delta_silver_path']=f'/dbfs{delta_silver_path}'


### 1. Using string matching and existing off-the-shelf vocabularies, create a simple pipeline to get rudimentary results.

First, we'll rely on existing vocabularies comprising of key words e.g: "kill my self", "want to die" etc to get preliminary results.

The vocbulary in this exercise is obtained from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4690390/

**About the dataset:** The dataset is a collection of reddit posts, taken from the work done by: 
- http://users.umiacs.umd.edu/~resnik/umd_reddit_suicidality_dataset.html
- https://github.com/hesamuel/goodbye_world

#### 1.1. Download resources and explore existing vocabularies

In [0]:
%sh

cd $delta_bronze_path

wget https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/databricks/python/healthcare_case_studies/data/suicide_pred_vocab_data.zip

unzip suicide_pred_vocab_data.zip

--2022-11-16 04:29:49-- https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/databricks/python/healthcare_case_studies/data/suicide_pred_vocab_data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-11-16 04:29:50 ERROR 404: Not Found.

Archive: suicide_pred_vocab_data.zip

In [0]:
# checking for files

data_vocab_path = f"{delta_bronze_path}suicide_pred_vocab_data/"

dbutils.fs.ls(data_vocab_path)

Out[3]: [FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_behavior_vocab.txt', name='suicide_behavior_vocab.txt', size=7654),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_det_data.csv/', name='suicide_det_data.csv/', size=0),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_others_vocab.txt', name='suicide_others_vocab.txt', size=2590),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_psychache_vocab.txt', name='suicide_psychache_vocab.txt', size=5569),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_trauma_vocab.txt', name='suicide_trauma_vocab.txt', size=5578)]

Taking a look at the vocabulary list

In [0]:
%sh

head /dbfs/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_behavior_vocab.txt

died
kill
love you
death
god
quiet
destiny
quiet
take medicine
quietly

#### 1.2. Creating a Spark NLP pipeline using textmatchers to find entities in the data

In [0]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols("sentence")\
  .setOutputCol("token")

text_matcher1 = TextMatcher().setInputCols("sentence","token").setOutputCol("textmatch_chunk1").setEntityValue("SUICIDE_BEHAVIOR")\
    .setEntities(data_vocab_path+"suicide_behavior_vocab.txt").setCaseSensitive(False).setMergeOverlapping(True)\
    .setBuildFromTokens(True)

text_matcher2 = TextMatcher().setInputCols("sentence","token").setOutputCol("textmatch_chunk2").setEntityValue("SUICIDE_PSYCHACHE")\
    .setEntities(data_vocab_path+"suicide_psychache_vocab.txt").setCaseSensitive(False).setMergeOverlapping(True)\
    .setBuildFromTokens(True)

text_matcher3 = TextMatcher().setInputCols("sentence","token").setOutputCol("textmatch_chunk3").setEntityValue("PAST_TRAUMA")\
    .setEntities(data_vocab_path+"suicide_trauma_vocab.txt").setCaseSensitive(False).setMergeOverlapping(True)\
    .setBuildFromTokens(True)

chunk_merger = ChunkMergeApproach()\
    .setInputCols(['textmatch_chunk1', 'textmatch_chunk2', 'textmatch_chunk3'])\
    .setOutputCol("all_chunks")

pipeline =  Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        text_matcher1, text_matcher2, text_matcher3,
        chunk_merger
    ]
)

p_model = pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

l_model = LightPipeline(p_model)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:499: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
 arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]

In [0]:
# Load the target data.

data = spark.read.csv(data_vocab_path+'suicide_det_data.csv').withColumnRenamed('_c0', 'text')
print (data.count())
data.show(3)


2667
+--------------------+
 text|
+--------------------+
I really don't wa...|
the answer is non...|
to whoever read t...|
+--------------------+
only showing top 3 rows

In [0]:
results = p_model.transform(data).collect()

#### 1.4. Analyzing Results

Even the basic textmatching approach works in general, but has the following challenges:
1. Lack of context - leading to too many false positives.
2. Less meaningful / incomplete chunks.

In [0]:
# random check
from sparknlp_display import NerVisualizer

displayHTML(NerVisualizer().display(results[500], 'all_chunks', return_html=True))

How to kill myself SUICIDE_BEHAVIOR - i'm looking for an easy way out PAST_TRAUMA . i'm a coward, i know. i can't PAST_TRAUMA take any more hearthache though. i need a quick way to kill myself SUICIDE_BEHAVIOR . would 750/800 mg of doxylamine succinate do the trick? if not is there some concoction of otc pills i can take? please help SUICIDE_BEHAVIOR , i'm desperate SUICIDE_BEHAVIOR .

In [0]:
# Checking on a piece of text
text = """My ex attempted suicide - she got way too attached and when i ghosted her i wanted to hate her , but i couldn’t after her attempt . so i hated myself ."""

results_single = l_model.fullAnnotate(text)[0]

from sparknlp_display import NerVisualizer

displayHTML(NerVisualizer().display(results_single, 'all_chunks', return_html=True))

My ex attempted suicide SUICIDE_BEHAVIOR - she got way too attached and when i ghosted her i wanted to hate PAST_TRAUMA her , but i couldn’t after PAST_TRAUMA her attempt . so i hated myself .

### 2. Now, we can use these results as pre-annotations and uploade on the Annotation Lab. Pre-annotations help reduce manual annotation time as the annotator does not need to annotate everything, but rather make corrections.



For this exercise, we are using John Snow Lab's Annotation Lab tool.

The Annotation Lab is a stand-alone web interface designed to be used and installed inside any organization's environment to protect data privacy. It can be easily installed on a single VM.

More details and instructions can be found here: https://nlp.johnsnowlabs.com/docs/en/alab/install#aws-marketplace. 

While the tasks and pre-annotations can be uploaded directly via web interface as well, we are leveraging the API module for convenience.

#### 2.1. Generate Pre-annotations using the Alab Module.

Initialize Alab module

In [0]:
# annotation lab module
from sparknlp_jsl.alab import AnnotationLab

alab = AnnotationLab()

In [0]:
# NOTE: "all_results" is the result of the pipeline after running on sample docs.
pre_annotations, summary = alab.generate_preannotations(all_results = results, document_column = 'document', ner_columns = ['all_chunks'])
pre_annotations[:10]

Processing 2667 Annotations.
Out[14]: [{'predictions': [{'created_username': 'model',
 'result': [{'from_name': 'label',
 'id': 'cE3y2gSYJF',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 27,
 'labels': ['SUICIDE_BEHAVIOR'],
 'start': 9,
 'text': "don't want to live"}},
 {'from_name': 'label',
 'id': 'h0i6MEicm7',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 111,
 'labels': ['SUICIDE_BEHAVIOR'],
 'start': 93,
 'text': "don't want to live"}},
 {'from_name': 'label',
 'id': 'MPoevO2Ic6',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 151,
 'labels': ['PAST_TRAUMA'],
 'start': 148,
 'text': 'why'}},
 {'from_name': 'label',
 'id': 'rGdXp3NGtU',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 170,
 'labels': ['PAST_TRAUMA'],
 'start': 165,
 'text': "can't"}},
 {'from_name': 'label',
 'id': 'sNBIuG9yCB',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 200,
 'labels': ['SUICIDE_BEHAVIOR'],
 'start': 196,
 'text': 'life'}},
 {'from_name': 'label',
 'id': 'a1k1NijDlG',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 257,
 'labels': ['SUICIDE_BEHAVIOR'],
 'start': 254,
 'text': 'cut'}},
 {'from_name': 'label',
 'id': 'HOfqqf4Qz9',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 298,
 'labels': ['SUICIDE_BEHAVIOR'],
 'start': 294,
 'text': 'will'}},
 {'from_name': 'label',
 'id': 'sApljsZO5K',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 312,
 'labels': ['PAST_TRAUMA'],
 'start': 307,
 'text': 'scars'}},
 {'from_name': 'label',
 'id': 'Rbj6yrF5pT',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 328,
 'labels': ['SUICIDE_PSYCHACHE'],
 'start': 320,
 'text': 'confused'}},
 {'from_name': 'label',
 'id': 'LPJgsK4G1y',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 409,
 'labels': ['SUICIDE_PSYCHACHE'],
 'start': 405,
 'text': 'mind'}},
 {'from_name': 'label',
 'id': 'aa90od9Y0D',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 453,
 'labels': ['PAST_TRAUMA'],
 'start': 449,
 'text': 'love'}},
 {'from_name': 'label',
 'id': 'd9arwIBgHJ',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 478,
 'labels': ['PAST_TRAUMA'],
 'start': 474,
 'text': 'love'}},
 {'from_name': 'label',
 'id': '7cs4MLwCgU',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 509,
 'labels': ['PAST_TRAUMA'],
 'start': 498,
 'text': 'hate myself'}}]}],
 'data': {'title': 'task_0',
 'text': "I really don't want to live anymore - i've never talked about this with anyone, but i really don't want to live anymore. because what is the point? why am i here? i can't do anything useful in my life until i become an adult, i ruin almost everything, i cut a lot and am always worried someone will see the scars. i get confused when i see people genuinely enjoying themselves and having fun. it blows my mind. how are they so happy? how can anyone love themselves? i don't love myself. i'm fact i hate myself. because i deserve it. what use am i to anyone? \n"},
 'id': 0},
 {'predictions': [{'created_username': 'model',
 'result': [{'from_name': 'label',
 'id': 'MEHkf5dD3d',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'end': 56,
 'labels': ['PAST_TRAUMA'],
 'start': 53,
 'text': 'why'}}]}],
 'data': {'title': 'task_1',
 'text': "the answer is none. i'm not of any use. and you know why? it's because whenever i've tried to be useful i mess it up. so"},
 'id': 1},
 {'predictions': [{'created_username': 'model', 'result': []}],
 'data': {'title': 'task_2',
 'text': 'to whoever read this all the way through'},
 'id': 2},
 {'predictions': [{'created_username': 'model',
 'result': [{'from_name': 'label',
 'id': 'TLfslpsK4M',
 'source': '$text',
 'to_name': 'text',
 'type': 'labels',
 'value': {'en

In [0]:
## What is summary? 
# While a user would know all the entities comming out of the NLP pipeline, listing all of them manually is laborious.
# The summary object helps identify how many types of entities, assertions, and relations are present.
# This saves the user from listing all labels individually; we'll use this object while setting project configuration.
summary

Out[15]: {'ner_labels': ['SUICIDE_BEHAVIOR', 'PAST_TRAUMA', 'SUICIDE_PSYCHACHE'],
 'assertion_labels': [],
 're_labels': []}

#### 2.2. Set Annotation Lab credentials and Create a New Project

In [0]:

# Set Credentials
username=''
password=''
client_secret=""
annotationlab_url=""

alab.set_credentials(

  # required: username
  username=username,

  # required: password
  password=password,

  # required: secret for you alab instance (every alab installation has a different secret)
  client_secret=client_secret,

  # required: http(s) url for you annotation lab
  annotationlab_url=annotationlab_url
)

In [0]:
# create new project
alab.create_project('suicide_detection')

Operation completed successfully. Response code: 201
Out[49]: {'project_name': 'suicide_detection'}

#### 2.3. Set project configuration (NER label tags, assertion classes, and relation tags)

In [0]:
# set configuration

## either manually define labels:
# alab.set_project_config(
#   project_name = 'suicide_detection',
#   ner_labels = ['SUICIDE_BEHAVIOR', 'SUICIDE_TRAUMA', 'SUICIDE_OTHERS', 'SUICIDE_PSYCHACHE']
# )

# OR use the summary object which already has all details

alab.set_project_config(
  project_name = 'suicide_detection',
  ner_labels = summary['ner_labels'],
  assertion_labels = summary['assertion_labels'],
  relations_labels = summary['re_labels']
)

Operation completed successfully. Response code: 201
Out[18]: {'messages': [{'message': 'Project config saved.', 'success': True}]}

#### 2.4. Upload pre-annotations to the newly created project

In [0]:
# Upload documents to Alab

alab.upload_preannotations(
  project_name = 'suicide_detection',
  preannotations = pre_annotations[:5]) # testing with 5 annotations

Uploading 5 preannotation(s).
Operation completed successfully. Response code: 201
Out[19]: {'completion_count': 0,
 'duration': 0.31545352935791016,
 'failed_count': 0,
 'ignored_count': 0,
 'prediction_count': 5,
 'task_count': 5,
 'task_ids': [1, 2, 3, 4, 5],
 'task_title_warning': 0,
 'updated_count': 0}

### 2.5. Annotate documents on Annotation Lab and make necessary corrections

### 3. Now, we can train an NER model using the annotations performed on the Annotation Lab

#### 3.1. Import the annotations from Annotation Lab and save as a JSON file.

In [0]:
exported_json = alab.get_annotations(
  project_name = 'suicide_detection', 
  output_name='result',
  save_dir=f"/dbfs/{delta_silver_path}")


Operation completed successfully. Response code: 200
Exported 132 Tasks
Annotations saved as /dbfs//FileStore/HLS/nlp/delta/silver//result.json.

In [0]:
print (delta_silver_path)
dbutils.fs.ls(delta_silver_path)

/FileStore/HLS/nlp/delta/silver/
Out[15]: [FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/silver/conll_demo.conll', name='conll_demo.conll', size=79866),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/silver/ner_model/', name='ner_model/', size=0),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/silver/result.json', name='result.json', size=366192)]

#### 3.2. Convert the JSON file to CoNLL format for training an NER model

In [0]:
alab.get_conll_data(spark, f"/dbfs/{delta_silver_path}result.json", output_name='conll_demo', save_dir=f"/dbfs/{delta_silver_path}")

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][ / ][ — ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][ / ][OK!]
Spark NLP LightPipeline is created
sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
Spark NLP LightPipeline is created
Attempting to process: Task ID# 1
Task ID# 1 is included
Attempting to process: Task ID# 2
Task ID# 2 is included
Attempting to process: Task ID# 3
Task ID# 3 is included
Attempting to process: Task ID# 4
Task ID# 4 is included
Attempting to process: Task ID# 5
Task ID# 5 is included
Attempting to process: Task ID# 6
Task ID# 6 is included
Attempting to process: Task ID# 7
Task ID# 7 is included
Attempting to process: Task ID# 8
Task ID# 8 is included
Attempting to process: Task ID# 9
Task ID# 9 is included
Attempting to process: Task ID# 10
Task ID# 10 is included
Attempting to process: Task ID# 11
Task ID# 11 is included
Attempting to process: Task ID# 12
Task ID# 12 is included
Attempting to process: Task ID# 13
Task ID# 13 is included
Attempting to process: Task ID# 14
Task ID# 14 is included
Attempting to process: Task ID# 15
Task ID# 15 is included
Attempting to process: Task ID# 16
Task ID# 16 is included
Attempting to process: Task ID# 17
Task ID# 17 is included
Attempting to process: Task ID# 18
Task ID# 18 is included
Attempting to process: Task ID# 19
Task ID# 19 is included
Attempting to process: Task ID# 20
Task ID# 20 is included
Attempting to process: Task ID# 21
Task ID# 21 is included
Attempting to process: Task ID# 22
Task ID# 22 is included
Attempting to process: Task ID# 23
Task ID# 23 is included
Attempting to process: Task ID# 24
Task ID# 24 is included
Attempting to process: Task ID# 25
Task ID# 25 is included
Attempting to process: Task ID# 26
Task ID# 26 is included
Attempting to process: Task ID# 27
Task ID# 27 is included
Attempting to process: Task ID# 28
Task ID# 28 is included
Attempting to process: Task ID# 29
Task ID# 29 is included
Attempting to process: Task ID# 30
Task ID# 30 is included
Attempting to process: Task ID# 31
Task ID# 31 is included
Attempting to process: Task ID# 32
Task ID# 32 is included
Attempting to process: Task ID# 33
Task ID# 33 is included
Attempting to process: Task ID# 34
Task ID# 34 is included
Attempting to process: Task ID# 35
Task ID# 35 is included
Attempting to process: Task ID# 36
Task ID# 36 is included
Attempting to process: Task ID# 37
Task ID# 37 is included
Attempting to process: Task ID# 38
Task ID# 38 is included
Attempting to process: Task ID# 39
Task ID# 39 is included
Attempting to process: Task ID# 40
Task ID# 40 is included
Attempting to process: Task ID# 41
Task ID# 41 is included
Attempting to process: Task ID# 42
Task ID# 42 is included
Attempting to process: Task ID# 43
Task ID# 43 is included
Attempting to process: Task ID# 44
Task ID# 44 is included
Attempting to process: Task ID# 45
Task ID# 45 is included
Attempting to process: Task ID# 46
Task ID# 46 is included
Attempting to process: Task ID# 47
Task ID# 47 is included
Attempting to process: Task ID# 48
Task ID# 48 is included
Attempting to process: Task ID# 49
Task ID# 49 is included
Attempting to process: Task ID# 50
Task ID# 50 is included
Attempting to process: Task ID# 51
Task ID# 51 is included
Attempting to process: Task ID# 52
Task ID# 52 is included
Attempting to process: Task ID# 53
Task ID# 53 is included
Attempting to process: Task ID# 54
Task ID# 54 is included
Attempting to process: Task ID# 55
Task ID# 55 is included
Attempting to process: Task ID# 56
Task ID# 56 is included
Attempting to process: Task ID# 57
Task ID# 57 is included
Attempting to process: Task ID# 58
Task ID# 58 is included
Attempting to process: Task ID# 59
Task ID# 59 is included
Attempting to process: Task ID# 60
Task ID# 60 is included
Attempting to process: Task ID# 61
T

In [0]:
dbutils.fs.ls(delta_silver_path)

Out[17]: [FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/silver/conll_demo.conll', name='conll_demo.conll', size=461135),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/silver/ner_model/', name='ner_model/', size=0),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/silver/result.json', name='result.json', size=366192)]

#### 3.3. Train NER Model

Load Data

In [0]:
conll_data = CoNLL().readDataset(spark, f"{delta_silver_path}conll_demo.conll")

conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| pos| label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
I know suicide is...|[{document, 0, 83...|[{document, 0, 83...|[{token, 0, 0, I,...|[{pos, 0, 0, MC, ...|[{named_entity, 0...|
i have zero hope ...|[{document, 0, 54...|[{document, 0, 54...|[{token, 0, 0, i,...|[{pos, 0, 0, NN, ...|[{named_entity, 0...|
the only things t...|[{document, 0, 11...|[{document, 0, 11...|[{token, 0, 2, th...|[{pos, 0, 2, DD, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

Look at label distribution

In [0]:

from pyspark.sql import functions as F

conll_data.select(F.explode(F.arrays_zip(conll_data.token.result,
                                         conll_data.label.result)).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth"))\
          .groupBy('ground_truth')\
          .count()\
          .orderBy('count', ascending=False)\
          .show(100,truncate=False)



+-------------------+-----+
ground_truth |count|
+-------------------+-----+
O |34041|
B-SUICIDE_BEHAVIOR |100 |
I-SUICIDE_BEHAVIOR |70 |
B-SUICIDE_PSYCHACHE|19 |
I-SUICIDE_PSYCHACHE|1 |
+-------------------+-----+

Select Embeddings

In [0]:
clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

Graph Builder to automatically generate a TensorFlow graph for training.

In [0]:
graph_folder_path = "/dbfs/ner/medical_ner_graphs"

ner_graph_builder = TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(20)\
    .setIsMedical(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

Ner Model with hyper-parameters

In [0]:
nerTagger = MedicalNerApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(35)\
  .setBatchSize(8)\
  .setRandomSeed(0)\
  .setVerbose(1)\
  .setLr(0.001)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setOutputLogsPath('dbfs:/ner/ner_logs')\
  .setUseBestModel(True)\
  .setGraphFolder('dbfs:/ner/medical_ner_graphs')\
  .setValidationSplit(0.2)
  # .setEnableMemoryOptimizer(True) #>> if you have a limited memory and a large conll file, you can set this True to train batch by batch       

ner_pipeline = Pipeline(stages=[
          clinical_embeddings,
          ner_graph_builder,
          nerTagger
 ])

Train

In [0]:
model = ner_pipeline.fit(conll_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: /dbfs/ner/medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 5, 'embeddings_dim': 200, 'nchars': 86, 'is_medical': True, 'lstm_size': 20}
ner_dl graph exported to /dbfs/ner/medical_ner_graphs/blstm_5_200_20_86.pb

Check logs

In [0]:
ls -l /dbfs/ner/ner_logs/

total 67
-rwxrwxrwx 1 root root 20821 Dec 10 20:28 MedicalNerApproach_46d751b42de6.log*
-rwxrwxrwx 1 root root 12365 Dec 9 06:25 MedicalNerApproach_6a40bb5d421d.log*
-rwxrwxrwx 1 root root 11402 Dec 8 10:00 MedicalNerApproach_7e4309ae2ec8.log*
-rwxrwxrwx 1 root root 22189 Dec 10 20:33 MedicalNerApproach_eade548198a2.log*

In [0]:
with open('/dbfs/ner/ner_logs/MedicalNerApproach_eade548198a2.log', 'r') as f_:
  lines = ''.join(f_)
print (lines)

Name of the selected graph: /local_disk0/tmp/sparknlp_tmp_4480777073595049037/blstm_5_200_20_86.pb
Training started - total epochs: 35 - lr: 0.001 - batch size: 8 - labels: 5 - chars: 83 - training examples: 1287


Epoch 1/35 started, lr: 0.001, dataset size: 1287


Epoch 1/35 - 5.77s - loss: 380.89035 - avg training loss: 2.3367505 - batches: 163
Quality on validation dataset (20.0%), validation examples = 257
time to finish evaluation: 0.79s
Total validation loss: 35.1186	Avg validation loss: 0.8167
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 0 12 0.0 0.0 0.0
B-SUICIDE_PSYCHACHE 0 0 6 0.0 0.0 0.0
B-SUICIDE_BEHAVIOR 0 0 17 0.0 0.0 0.0
tp: 0 fp: 0 fn: 35 labels: 3
Macro-average prec: 0.0, rec: 0.0, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 2/35 started, lr: 9.950249E-4, dataset size: 1287


Epoch 2/35 - 3.97s - loss: 156.12029 - avg training loss: 0.9577932 - batches: 163
Quality on validation dataset (20.0%), validation examples = 257
time to finish evaluation: 0.33s
Total validation loss: 30.5197	Avg validation loss: 0.7098
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 0 12 0.0 0.0 0.0
B-SUICIDE_PSYCHACHE 0 0 6 0.0 0.0 0.0
B-SUICIDE_BEHAVIOR 0 0 17 0.0 0.0 0.0
tp: 0 fp: 0 fn: 35 labels: 3
Macro-average prec: 0.0, rec: 0.0, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 3/35 started, lr: 9.90099E-4, dataset size: 1287


Epoch 3/35 - 3.92s - loss: 129.23996 - avg training loss: 0.7928832 - batches: 163
Quality on validation dataset (20.0%), validation examples = 257
time to finish evaluation: 0.34s
Total validation loss: 24.8755	Avg validation loss: 0.5785
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 0 12 0.0 0.0 0.0
B-SUICIDE_PSYCHACHE 0 0 6 0.0 0.0 0.0
B-SUICIDE_BEHAVIOR 0 0 17 0.0 0.0 0.0
tp: 0 fp: 0 fn: 35 labels: 3
Macro-average prec: 0.0, rec: 0.0, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 4/35 started, lr: 9.852217E-4, dataset size: 1287


Epoch 4/35 - 3.99s - loss: 120.7001 - avg training loss: 0.74049145 - batches: 163
Quality on validation dataset (20.0%), validation examples = 257
time to finish evaluation: 0.34s
Total validation loss: 23.6999	Avg validation loss: 0.5512
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 0 12 0.0 0.0 0.0
B-SUICIDE_PSYCHACHE 0 0 6 0.0 0.0 0.0
B-SUICIDE_BEHAVIOR 0 0 17 0.0 0.0 0.0
tp: 0 fp: 0 fn: 35 labels: 3
Macro-average prec: 0.0, rec: 0.0, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 5/35 started, lr: 9.803922E-4, dataset size: 1287


Epoch 5/35 - 3.95s - loss: 110.61422 - avg training loss: 0.67861485 - batches: 163
Quality on validation dataset (20.0%), validation examples = 257
time to finish evaluation: 0.34s
Total validation loss: 21.9626	Avg validation loss: 0.5108
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 0 12 0.0 0.0 0.0
B-SUICIDE_PSYCHACHE 0 0 6 0.0 0.0 0.0
B-SUICIDE_BEHAVIOR 0 0 17 0.0 0.0 0.0
tp: 0 fp: 0 fn: 35 labels: 3
Macro-average prec: 0.0, rec: 0.0, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 6/35 started, lr: 9.756098E-4, dataset size: 1287


Epoch 6/35 - 3.92s - loss: 100.92268 - avg training loss: 0.61915755 - batches: 163
Quality on validation dataset (20.0%), validation examples = 257
time to finish evaluation: 0.33s
Total validation loss: 34.0493	Avg validation loss: 0.7918
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 0 12 0.0 0.0 0.0
B-SUICIDE_PSYCHACHE 0 0 6 0.0 0.0 0.0
B-SUICIDE_BEHAVIOR 0 0 17 0.0 0.0 0.0
tp: 0 fp: 0 fn: 35 labels: 3
Macro-average prec: 0.0, rec: 0.0, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 7/35 started, lr: 9.7087387E-4, dataset size: 1287


Epoch 7/35 - 3.93s - loss: 97.14512 - avg training loss: 0.5959823 - batches: 163
Quality on validation dataset (20.0%), validation examples = 257
time to finish evaluation: 0.34s
Total validation loss: 20.2453	Avg validation loss: 0.4708
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 0 12 0.0 0.0 0.0
B-SUICIDE_PSYCHACHE 0 0 6 0.0 0.0 0.0
B-SUICIDE_BEHAVIOR 0 0 17 0.0 0.0 0.0
tp: 0 fp: 0 fn:

Save the model to disk, load and test on the new model

In [0]:
#save model
model.stages[-1].write().overwrite().save(delta_silver_path+'ner_model')

In [0]:
# use model in a prediction pipeline

documentAssembler = DocumentAssembler()\
            .setInputCol("text")\
            .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel().pretrained('sentence_detector_dl_healthcare', 'en', 'clinical/models')\
           .setInputCols(["document"])\
           .setOutputCol("sentence")

tokenizer = Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")
embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', 'en', 'clinical/models') \
      .setInputCols("sentence", "token") \
      .setOutputCol("embeddings")

#load the trained ner model
ner_model =MedicalNerModel().load(delta_silver_path+'ner_model')\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_tags")

#ner converter ner jsl enriched
ner_chunk = NerConverterInternal()\
    .setInputCols(['sentence', 'token', 'ner_tags']) \
    .setOutputCol('ner_chunk')


pipeline=Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        ner_model,
        ner_chunk,
        
         ])
empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)
light_model = LightPipeline(model)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
# Checking on a piece of text
text = """My ex attempted suicide - she got way too attached and when i ghosted her i wanted to hate her , but i couldn’t after her attempt . so i hated myself ."""

results_single = light_model.fullAnnotate(text)[0]

from sparknlp_display import NerVisualizer

displayHTML(NerVisualizer().display(results_single, 'ner_chunk', return_html=True))

My ex attempted suicide SUICIDE_BEHAVIOR - she got way too attached and when i ghosted her i wanted to hate her , but i couldn’t after her attempt . so i hated myself SUICIDE_BEHAVIOR .